В этом ноутбуке кортежи мнений предсказываются с помощью модели Mistral Large 2 (mistral-large-latest).

Для использования необходим Mistral API key, который можно получить бесплатно.

### Проверка ###

In [37]:
import requests
import ast
import json
import os
import itertools
from tqdm import tqdm
import pandas as pd
from collections import defaultdict
import random
from random import choices
import time

api_key = "YOUR_API_KEY"
model = "mistral-large-latest"
url = "https://api.mistral.ai/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}
SEED = 42

random.seed(SEED)

In [38]:
response = requests.get(url)
response.headers

{'Date': 'Wed, 27 Nov 2024 02:11:13 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '96', 'Connection': 'keep-alive', 'www-authenticate': 'Key', 'access-control-allow-origin': '*', 'x-kong-response-latency': '1', 'x-kong-request-id': '83d989298e086e0226bcaf4e6adcf534', 'CF-Cache-Status': 'DYNAMIC', 'Server': 'cloudflare', 'CF-RAY': '8e8e9616af716f72-CDG', 'alt-svc': 'h3=":443"; ma=86400'}

In [39]:
class MistralModel():
    def __init__(self, model, temp = 0.2, top_p = 0.9, max_tokens = 256, seed = 42):
        self.model = model
        self.temp = temp
        self.top_p = top_p
        self.max_tokens = max_tokens
        self.seed = 42
    def prompt(self, text):
        prompt = {'model': self.model,
                  'random_seed': self.seed,
              'temperature': self.temp,
              'top_p': self.top_p,
              'max_tokens': self.max_tokens,
              'messages': [{'role':'user', 'content': text}]
             }
        return prompt
    
    def structured_prompt(self, instruction, text):
        prompt = {'model': self.model,
                  'random_seed': self.seed,
              'temperature': self.temp,
              'top_p': self.top_p,
              'max_tokens': self.max_tokens,
              'messages': [{'role':'system', 'content': instruction},
                          {'role':'user', 'content': text}]
             }
        return prompt

mistral = MistralModel(model, max_tokens = 512)

instruction = "ответь на вопрос"
text = "тебя создали французы?"
response = requests.post(url, headers=headers, json=mistral.prompt(text))
print(f'Вопрос: {text}')
print(f"Ответ: {response.json()['choices'][0]['message']['content']}")

Вопрос: тебя создали французы?
Ответ: Нет, меня создала Mistral AI, передовая французская компания в области искусственного интеллекта.


### Подготовка данных ###

In [29]:
import requests, zipfile, io

url = 'https://raw.githubusercontent.com/rossyaykin/RuOpinionNE/refs/heads/main/src/src.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall('')

from src.utils import load_jsonl, save_jsonl, str2list, dict2tuple, extract_tuple, df2structure, form_prompt

In [30]:
train_path = "full.jsonl"
test_path = "validation.jsonl"

url = 'https://raw.githubusercontent.com/rossyaykin/RuOpinionNE/refs/heads/main/data/full.jsonl'
train = load_jsonl(url, train_path)
url = 'https://raw.githubusercontent.com/rossyaykin/RuOpinionNE/refs/heads/main/data/validation.jsonl'
test = load_jsonl(url, test_path)

print(len(train), len(test))

2556 1316


### Определения ###

In [31]:
class Runner():
    def __init__(self, model, url, headers, train, test, n_shots = 5, sleeptime = 2):
        self.model = model
        self.url = url
        self.headers = headers
        self.train = train
        self.test = test
        self.n_shots = n_shots
        self.sleeptime = sleeptime
    
    def run(self):
        results = list()
        for entry in tqdm(self.test):
            time.sleep(self.sleeptime)
            examples = [dict2tuple(x) for x in choices(self.train, k = n_shots)]
            prompt = form_prompt(examples, entry['text'])
            response = requests.post(self.url,
                                     headers=self.headers,
                                     json=self.model.prompt(prompt))
            result = []
            if response.status_code == 200:
                response = response.json()['choices'][0]['message']['content']
                try:
                    result = ast.literal_eval(response)
                except (SyntaxError, ValueError):
                    print(f'bad response, iteration:{len(results)}')
            else:
                print(f'bad response, iteration:{len(results)}') 
            results.append((entry['sent_id'],
                            entry['text'],
                            dict2tuple(entry)[1], # gold opinions
                            result)) # pred opinions
        return results

def get_path(temp, n_shots):
    path = f'./results/Mistral/Mistral_bl_{n_shots}shot_{temp}temp'
    # returns full path but without ".csv"
    return path

def save(dataframe, path, raw = True):
    outdir, outname = '/'.join(path.split('/')[:-1]), path.split('/')[-1]
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    if raw:
        dataframe.to_csv(f'{path}_raw.csv', index = False)
    else:
        dataframe.to_csv(f'{path}.csv', index = False)

### Тест на одном примере ###

In [40]:
n_shots = 5
examples = [dict2tuple(x) for x in train[:n_shots]]
text, target = dict2tuple(train[n_shots])

sample_prompt = form_prompt(examples, text)
print(f'--Промпт--:\n\n{sample_prompt}')

--Промпт--:

Ты эксперт в оценке тональности.
Тебе нужно найти все негативные и позитивные отношения между сущностями в тексте и вывести их в следующем формате:
[источник отношения, объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Если источником отношения является автор, то пиши:
['AUTHOR', объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Если выраженного источника нет, то пиши:
['NULL', объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Допустимо вернуть пустой ответ:
[]
Не нужно давать пояснений к ответу.
Примеры
Текст: Президент Башкирии Муртаза Рахимов в очередной раз решил поменять главу своей администрации.
Ответ: [['Муртаза Рахимов', 'главу своей администрации', 'поменять', 'NEG']]
Текст: Вчера он уволил Азамата Сагитова, который возглавил башкирскую администрацию год назад после вынужденной отставки Радия Хабирова, сейчас занимающего пост заместителя начальника управления президента РФ по внутренней полит

In [41]:
response = requests.post(url, headers=headers, json=mistral.prompt(sample_prompt))
response = response.json()['choices'][0]['message']['content']
result = ast.literal_eval(response)

print(f'--Текст--:\n{text}')
print(f'--Таргет--:\n{target}')
print(f'--Предикт--:\n{extract_tuple(result)}')

--Текст--:
Этому назначению предшествовал громкий скандал, сопровождавший историю отставки прежнего главы администрации Радия Хабирова.
--Таргет--:
[['NULL', 'Радия Хабирова', 'громкий скандал', 'NEG']]
--Предикт--:
[['NULL', 'Радия Хабирова', 'скандал', 'NEG'], ['NULL', 'Радия Хабирова', 'отставки', 'NEG']]


### Инференс ###

In [6]:
%%time
params = {'temp': 0.4,
          'seed': SEED,
          'top_p': 0.9,
          'max_tokens': 512}
mistral = MistralModel(model, **params)
n_shots = 15
runner = Runner(mistral, url, headers, train, val, n_shots)

path = get_path(mistral.temp, n_shots)
result = runner.run()

 90%|████████▉ | 1181/1316 [1:20:47<10:01,  4.45s/it]

bad response, iteration:1180


100%|██████████| 1316/1316 [1:30:08<00:00,  4.11s/it]

CPU times: total: 6min 21s
Wall time: 1h 30min 8s


In [7]:
print(path)
print(result[7])

./results/Mistral/Mistral_bl_15shot_0.4temp
(7, 'Напомним, на казахстанском пограничном временном посту "Арканкерген" были обнаружены обгоревшие тела 14 пограничников и 1 егеря.', [], [])


### Результаты ###

In [8]:
output = pd.DataFrame(result, columns = ['sent_id', 'text', 'target', 'pred'])
output.head()

,sent_id,text,target,pred
0,0,В числе участников президентской борьбы есть о...,[],[]
1,1,"Кандидатке на пост президента 54 года, она род...",[],[]
2,2,"Сама женщина заявила, что встречаться с сыном ...",[],"[[NULL, женщина, встречаться с сыном ей пришло..."
3,3,"Они снимали нас все эти 5 минут, что длилось с...",[],"[[NULL, Влад, ничего лишнего, NEG]]"
4,4,"Кроме того, по словам женщины на щеке сына Све...",[],"[[NULL, сына, синяк, NEG]]"


In [9]:
print(path)
save(output, path)

./results/Mistral/Mistral_bl_15shot_0.4temp


In [10]:
output = pd.DataFrame([(x[0], x[1], x[2], str2list(extract_tuple(x[3]))) for x in result],
                      columns = ['sent_id', 'text', 'target', 'pred'])
output.head()

,sent_id,text,target,pred
0,0,В числе участников президентской борьбы есть о...,[],[]
1,1,"Кандидатке на пост президента 54 года, она род...",[],[]
2,2,"Сама женщина заявила, что встречаться с сыном ...",[],"[[NULL, женщина, встречаться с сыном ей пришло..."
3,3,"Они снимали нас все эти 5 минут, что длилось с...",[],"[[NULL, Влад, ничего лишнего, NEG]]"
4,4,"Кроме того, по словам женщины на щеке сына Све...",[],"[[NULL, сына, синяк, NEG]]"


In [11]:
save(output, path, raw = False)

### csv to jsonl ###

In [13]:
final = df2structure(output)
final[3]

{'sent_id': 3,
 'text': 'Они снимали нас все эти 5 минут, что длилось свидание, чтобы Влад ничего лишнего мне не сказал.',
 'opinions': [{'Source': [['NULL'], ['0:0']],
   'Target': [['Влад'], ['61:65']],
   'Polar_expression': [['ничего лишнего'], ['66:80']],
   'Polarity': 'NEG'}]}

In [14]:
save_jsonl(final, path)